## Week 4 - Implement and query stored procedures

In this exercise you must create a pool of connections and get a connection from the pool to implement the stored procedures to complete the following tasks:

Establish a connection by importing MySQLConnectionPool and creating a pool with two connections.

Create and call a stored procedure named PeakHours that identifies the peak, or busiest hour, for the restaurant based on the number of bookings.

Create and call a stored procedure named GuestStatus that outputs status of each guest’s order based on which employee is assigned to the order.

Stored procedures are created to carry out routine operations on MySQL databases. They are consistent and make sure that the written SQL queries in the procedures are executed in the same way every time you call the stored procedure.

A stored procedure is created only once, and you store it in the MySQL database. You can call the stored procedures as many times as you need in your Python-based application.

The tasks and the steps that you must follow to complete each one are as follows:

## Task 1: : Establish a connection 

Create a database connection pool with three connections available for the users to connect. You need to import MySQLConnectionPool class and pass the following arguments: 

Step one: Import MySQLConnectionPool

Step two: Import Error

Step three: Create a pool named pool_a with two connections. Use a try-except block to handle any possible errors. 

Your database configurations are as follows:

```Python
dbconfig = { 
    "database" : "name_of_the_little_lemon_database",  
    "user" : "your_username", 
    "password" : "your_password" 
} 
```
Step four: Obtain a connection from pool_a and create a cursor object to communicate with the database.

In [1]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig = {
    "database":"little_lemon",
    "user" : "root",
    "password" : "root"
}

try:
    pool = MySQLConnectionPool(pool_name = "pool_a",
                           pool_size = 2, #default is 5
                           **dbconfig)
    print("The connection pool is created with a name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)

except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)
    
print("Getting a connection from the pool.")
connection = pool.get_connection()

# Create cursor object to communicate with entire MySQL database
print("Creating a cursor object.")
cursor = connection.cursor()


The connection pool is created with a name:  pool_a
The pool size is: 2
Getting a connection from the pool.
Creating a cursor object.


## Task 2: Implement a stored procedure called PeakHours

Step one: Write a SQL CREATE PROCEDURE query for PeakHours

Use HOUR to extract the hour part from the BookingSlot.

Use COUNT on hour to count the number of bookings.

Use GROUP BY on booking hour.

Use ORDER BY on the number of bookings in descending order.

Step two: Run the stored procedure query by invoking execute module on the cursor.

Step three: Invoke callproc to call the stored procedure.

Step four: Fetch the results in a variable called dataset.

Step five: Extract the names of the columns.

Step six: Print the names of the columns.

Step seven: Print the sorted data using for loop. 

In [3]:
stored_procedure_query="""
CREATE PROCEDURE PeakHours()

BEGIN

SELECT HOUR(BookingSlot) AS Hour, 
COUNT(BookingID) AS NoBooking 
FROM Bookings 
GROUP BY Hour 
ORDER BY NoBooking DESC;

END

"""

# Execute the query
cursor.execute("DROP PROCEDURE IF EXISTS PeakHours;")
cursor.execute(stored_procedure_query)

#********************************************#

# Call the stored procedure with its name
cursor.callproc("PeakHours")

# Retrieve recrods in "dataset"
results = next( cursor.stored_results() )
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data 
for data in dataset:
    print(data)

['Hour', 'NoBooking']
(19, 2)
(15, 1)
(17, 1)
(18, 1)
(20, 1)


## Task 3: Implement a stored procedure GuestStatus

Step one: Write a SQL CREATE PROCEDURE query for GuestStatus.

Step two: Combine the guest’s first and last name from the booking column using CONCAT. 

Step three: Use CASE to implement the following statuses for each guest’s order:

If the Role in the Employee table is Manager or Assistant Manager then the guest’s order status is Ready to pay

If the Role in the Employee table is Head Chef then the status is Ready to serve

If the Role in the Employee table is Assistant Chef then the status is Preparing Order

If the Role in the Employee table is Head Waiter then the status is Order served

Step four: LEFT JOIN Bookings table with Employees ON EmployeeID

Step five: Run the stored procedure query by invoking execute module on the cursor.

Step six: Invoke callproc to call the stored procedure.

Step seven: Fetch the results in a variable called dataset.

Step eight: Extract the names of the columns.

Step nine: Print the names of the columns.

Step ten: Print the sorted data using for loop.

Step eleven: Close the connection to return it back to the pool.

In [4]:
stored_procedure_query="""
CREATE PROCEDURE GuestStatus()

BEGIN

SELECT Bookings.BookingID AS OrderNumber,
CONCAT(GuestFirstName, " ", GuestLastName) AS GuestName, 
Role AS Employee,

CASE
WHEN Role IN ("Manager","Assistant Manager") THEN "Ready to pay"
WHEN Role = "Head Chef" THEN "Ready to serve"
WHEN Role = "Assistant Chef" THEN "Preparing Order"
WHEN Role = "Head Waiter" THEN "Order served"
ELSE "Pending"
END AS Status

FROM Bookings LEFT JOIN Employees 
ON Bookings.EmployeeID = Employees.EmployeeID;

END

"""

# Execute the query
cursor.execute(stored_procedure_query)

#********************************************#

# Call the stored procedure with its name
cursor.callproc("GuestStatus")

# Retrieve recrods in "dataset"
results = next( cursor.stored_results() )
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data 
for data in dataset:
    print(data)
    


['OrderNumber', 'GuestName', 'Employee', 'Status']
(1, 'Anna Iversen', 'Manager', 'Ready to pay')
(2, 'Joakim Iversen', 'Manager', 'Ready to pay')
(3, 'Vanessa McCarthy', 'Head Chef', 'Ready to serve')
(4, 'Marcos Romero', 'Assistant Chef', 'Preparing Order')
(5, 'Hiroki Yamane', 'Assistant Manager', 'Ready to pay')
(6, 'Diana Pinto', 'Head Waiter', 'Order served')


In [5]:
# Let's close the cursor and the connection
if connection.is_connected():
    cursor.close()
    print("The cursor is closed.")
    connection.close()
    print("MySQL connection is closed.")
else:
    print("Connection is already closed")
    

The cursor is closed.
MySQL connection is closed.
